## BTSLOG分析脚本
---
**环境**：
- Python 3.5+

**功能**
- 根据自定义规则过滤BTSLog

**历史**
- 0.01: 2016/09/30

In [1]:
import os
import re
import glob
import time

# 读取文件返回文件每一行
def lines(file):
    for line in file:
        yield line

# 获取SYSLOG列表
def filelist(path):
    os.chdir(path)
    file_list = glob.glob(r"./SYSLOG_*.LOG")
    return file_list

# Parser类
class Parser:
    def __init__(self):
        self.rules = []
        self.start_rule = ""
        self.end_rule = ""
        self.start_flag = False  # once start point matched, set this flag to true
        self.end_flag = False    # once end point matched, set this flag to false
        self.mid_flag = False    # the reason to introduce this flag is to print the last line which is matched with end point

    # 设置log分析的起始点和结束点    
    def setFilter(self, start_rule=r'.*', end_rule=r'^9999999999999'):
        'Set start point and end point which are for pasring'
        self.start_rule = start_rule
        self.end_rule = end_rule
    
    # 只解析起始点和结束点之间的log，所以要找到log中的起始点和结束点并标记
    def applyFilter(self, line):
        'Only to parse the logs between start point and end point'
        if self.mid_flag:
            self.end_flag = True
        if not self.start_flag:
            start = re.search(self.start_rule, line)
            if start:
                self.start_flag = True
        
        if (self.start_flag) and (not self.end_flag):
            end = re.search(self.end_rule, line)
            if end:
                self.mid_flag = True
    
    # 正则表达式表示的过滤规则
    def addRule(self, rule):
        'add reg pattern'
        self.rules.append(rule) 
    
    # 对Log应用过滤规则
    def applyRule(self, line):
        'apply reg pattern for paring log'
        # 如果Log中一行匹配多了个rule，为了避免重复打印，设置flag
        flag = False        
        for rule in self.rules:
            if not flag:
                search_result = re.search(rule, line, re.I|re.M)
                if (search_result):
                    flag = True
                    print(line.strip())     
    
    # 根据规则解析Log
    def parse(self, file):
        'parsing logs between start point and end point as configured'
        for line in lines(file):
            self.applyFilter(line)
            if (self.start_flag) and (not self.end_flag):
                self.applyRule(line)
        
        # set these flag to initial value for next log parsing
        self.start_flag = False
        self.end_flag = False
        self.mid_flag = False

# BTSLog解析子类
class BTSLogParser(Parser):
    def __init__(self):
        Parser.__init__(self)

In [2]:
LOG_PATH = r'D:\userdata\anliu\Desktop\log\BTSLogs' 

parser = BTSLogParser()  
parser.addRule(r'MAC_USER_DELETE')
parser.addRule(r'UecUeRelease')

# set start point and end point
start_point = r".*"
end_point = r'^020000'
parser.setFilter(start_point,end_point)

flist = filelist(LOG_PATH)
for file in flist:      
    print("Parsing log '%s' started......" %file.split("\\")[1])
    print("*"*60)
    with open(file) as f:  
        parser.parse(f)
    print("*"*60)
    print("Parsing Log '%s' completed!\n\n"%file.split('\\')[1])      

Parsing log 'SYSLOG_190.LOG' started......
************************************************************
007838 08.09 21:30:35.959  [192.168.255.129]  a7 FSP-2273 <2004-01-01T00:20:39.096803Z> 13-EmDispatch INF/LTE L2/L2Manager/cBr6EDFu628x6ElF/MAC_USER_DELETE_REQ: ueId=1576, cellId=11
007954 08.09 21:30:35.975  [192.168.255.129]  fc FSP-220D-6-UECexe <2004-01-01T00:20:39.081520Z> D34-EuUec::UeContex DBG/UEC/UecUeRelease.cpp#1122 [ueContextId:1576 cellId:35851] StateIdle() payLoadId:0x3110
007955 08.09 21:30:35.975  [192.168.255.129]  fd FSP-220D-6-UECexe <2004-01-01T00:20:39.081538Z> D34-EuUec::UeContex DBG/UEC/UecUeRelease.cpp#735 [ueContextId:1576 cellId:35851] validateUeConnectionRelease() payloadId:UEC_UEM_UE_CONNECTION_RELEASE cellId:35851 releaseReason:13
007994 08.09 21:30:35.990  [192.168.255.129]  03 FSP-220D-6-UECexe <2004-01-01T00:20:39.081618Z> D34-EuUec::UeContex DBG/UEC/UecUeRelease.cpp#854 [ueContextId:1576 cellId:35851] sendS1ApUeContextReleaseRequest() forwardEvent: TU